Gerekli Kütüphanelerin Yüklenmesi

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

Veri Setinin Yüklenmesi ve Hazırlanması.

In [ ]:
# Verilerin bulunduğu dizin
base_dir = '/kaggle/input/a-large-scale-fish-dataset/Fish_Dataset'

# Resim boyutları
IMG_WIDTH, IMG_HEIGHT = 150, 150  
BATCH_SIZE = 32


Veri Artırma.

In [ ]:
# Eğitim ve doğrulama veri artırma
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  
)

In [ ]:
# Eğitim verilerini yükleme
train_data = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'Fish_Dataset'),  
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'  
)  

In [ ]:
# Doğrulama verilerini yükle
val_data = train_datagen.flow_from_directory(
    os.path.join(base_dir, 'Fish_Dataset'),
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'  
)

Modelin Oluşturulması.

In [ ]:
# Modeli oluştur
model = Sequential()
model.add(Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3)))  # Giriş katmanı


In [ ]:
# Flatten katmanı
model.add(Flatten())  # Resmi düzleştir

In [ ]:
# Tam bağlantılı katmanlar
model.add(Dense(128, activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(9, activation='softmax'))  # Çıkış katmanı


Modelin Derlenmesi.

In [ ]:
# Modeli derlemek
model.compile(loss='categorical_crossentropy',  
              optimizer=Adam(learning_rate=0.0001),  
              metrics=['accuracy'])


Modelin Eğitilmesi.

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
# Modeli eğit
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=15,
    callbacks=[early_stopping]
)


Sonuçların Görselleştirilmesi.

In [ ]:
# Eğitim sürecinin görselleri
plt.plot(history.history['accuracy'], label='Eğitim Doğruluğu')
plt.plot(history.history['val_accuracy'], label='Doğrulama Doğruluğu')
plt.title('Model Doğruluğu')
plt.ylabel('Doğruluk')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.title('Model Kaybı')
plt.ylabel('Kayıp')
plt.xlabel('Epoch')
plt.legend()
plt.show()